# 1. 데이터 불러오고 범주형 변수 처리

In [4]:
import pandas as pd


# 1. CSV 데이터 로드
df = pd.read_csv('./results/2nd-dataset_서울 송파구_20250602_110023.csv')

# 2. 불필요한 컬럼 제거
# 'date'와 'region_code'는 현재 모델링에 직접적으로 필요하지 않으므로 제거합니다.
df = df.drop(columns=['date', 'region_code'])

# 3. 범주형 변수 처리 (원-핫 인코딩)
# 'sports_types', 'weather_condition', 'weekday' 컬럼을 원-핫 인코딩합니다.
# 'drop_first=True'는 다중 공선성(multicollinearity) 문제를 피하기 위해 첫 번째 카테고리를 제거합니다.
# 예를 들어 '맑음', '흐림', '비/눈'이 있다면 '맑음' 컬럼을 만들지 않고 '흐림', '비/눈' 컬럼만 만듭니다.
# '흐림'이 0이고 '비/눈'이 0이면 자동으로 '맑음'임을 알 수 있기 때문입니다.
df_encoded = pd.get_dummies(df, columns=['sports_types', 'weather_condition', 'weekday'], drop_first=True)

# 4. 피처(Feature)와 타겟(Target) 분리
# 'accident_count'가 우리가 예측하려는 타겟(y)입니다.
y = df_encoded['accident_count']
# 나머지 모든 컬럼은 예측에 사용될 피처(X)입니다.
X = df_encoded.drop(columns=['accident_count'])

print("--- 전처리된 데이터 (피처 X) 상위 5행 ---")
print(X.head())
print("\n--- 타겟 데이터 (y) 상위 5행 ---")
print(y.head())
print("\n--- 전처리 후 X 데이터의 컬럼 목록 ---")
print(X.columns)



--- 전처리된 데이터 (피처 X) 상위 5행 ---
   game_count  is_post_season  temperature  precipitation  snow_depth  \
0           1               0         -2.0            0.0         0.0   
1           0               0         -5.2            0.0         0.0   
2           0               0         -6.2            0.0         0.0   
3           1               0         -3.5            0.0         0.0   
4           1               0         -3.6            0.0         0.0   

   is_holiday  sports_types_농구,야구  sports_types_야구  sports_types_없음  \
0           1               False            False            False   
1           0               False            False             True   
2           0               False            False             True   
3           0               False            False            False   
4           0               False            False            False   

   weather_condition_맑음  weather_condition_비/눈  weather_condition_흐림  \
0                  True         

# 2. 데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split

# X, y가 정의되어 있는지 확인합니다.
if 'X' not in locals() or 'y' not in locals():
    raise ValueError("X 또는 y 데이터가 정의되지 않았습니다. 이전 단계를 먼저 실행해주세요.")

# 데이터를 훈련 세트와 테스트 세트로 분할합니다.
# test_size=0.2는 전체 데이터의 20%를 테스트 세트로 사용하겠다는 의미입니다.
# random_state는 난수 발생 시드를 고정하여, 매번 같은 결과를 얻을 수 있도록 합니다. 실험의 재현성을 유지
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"훈련 세트 크기 (X_train): {X_train.shape}")
print(f"테스트 세트 크기 (X_test): {X_test.shape}")
print(f"훈련 세트 크기 (y_train): {y_train.shape}")
print(f"테스트 세트 크기 (y_test): {y_test.shape}")

훈련 세트 크기 (X_train): (584, 18)
테스트 세트 크기 (X_test): (147, 18)
훈련 세트 크기 (y_train): (584,)
테스트 세트 크기 (y_test): (147,)


# 3. 선형회귀

In [ ]:
from sklearn.linear_model import LinearRegression

# 데이터가 정의되어 있는지 다시 확인합니다.
if 'X_train' not in locals() or 'y_train' not in locals():
    raise ValueError("X_train 또는 y_train 데이터가 정의되지 않았습니다. 이전 단계를 먼저 실행해주세요.")

# 1. LinearRegression 모델 객체 생성
linear_model = LinearRegression()

# 2. 훈련 세트(X_train, y_train)를 사용하여 모델 학습
print("선형 회귀 모델 학습 중...")
linear_model.fit(X_train, y_train)
print("선형 회귀 모델 학습 완료!")

# 학습된 모델의 계수(coefficients)와 절편(intercept) 확인
print(f"\n모델의 절편 (Intercept): {linear_model.intercept_:.2f}")
print("\n모델의 계수 (Coefficients):")
# 각 피처(컬럼)에 해당하는 계수 값을 출력하여 어떤 피처가 사고 건수에 영향을 미치는지 확인합니다.
for feature, coef in zip(X_train.columns, linear_model.coef_):
    print(f"  {feature}: {coef:.2f}")

선형 회귀 모델 학습 중...
선형 회귀 모델 학습 완료!

모델의 절편 (Intercept): 6.66

모델의 계수 (Coefficients):
  game_count: -0.05
  is_post_season: 0.05
  temperature: 0.05
  precipitation: -0.01
  snow_depth: 0.01
  is_holiday: -1.77
  sports_types_농구,야구: -0.82
  sports_types_야구: 0.10
  sports_types_없음: 0.64
  weather_condition_맑음: 0.46
  weather_condition_비/눈: 0.18
  weather_condition_흐림: -0.40
  weekday_목: -0.32
  weekday_수: -0.57
  weekday_월: -0.96
  weekday_일: -0.63
  weekday_토: 1.54
  weekday_화: -0.04


### 각 수치가 양으로 차이가 벌어진다면 y(교통사고 건수)에 대해 해당 특성이 영향을 많이 받는다는 의미

# 포아송 회귀

In [7]:
import statsmodels.api as sm

# X_train, y_train은 이전 단계에서 분할된 훈련 데이터입니다.
# 데이터가 정의되어 있는지 다시 확인합니다.
if 'X_train' not in locals() or 'y_train' not in locals():
    raise ValueError("X_train 또는 y_train 데이터가 정의되지 않았습니다. 이전 단계를 먼저 실행해주세요.")

# statsmodels는 기본적으로 절편(intercept)을 포함하지 않으므로, 명시적으로 추가해 줍니다.
# 'add_constant' 함수를 사용하여 X_train에 상수 항(constant, 즉 절편)을 추가합니다.
X_train_poisson = sm.add_constant(X_train)
X_train_poisson = X_train_poisson.astype(float)
y_train = y_train.astype(float)

# 1. Poisson 모델 정의
# sm.Poisson(종속변수, 독립변수) 형태로 모델을 정의합니다.
poisson_model = sm.Poisson(y_train, X_train_poisson)

# 2. 훈련 세트(X_train_poisson, y_train)를 사용하여 모델 학습
print("포아송 회귀 모델 학습 중...")
poisson_results = poisson_model.fit() # 학습된 모델의 결과는 'results' 객체에 저장됩니다.
print("포아송 회귀 모델 학습 완료!")

# 학습된 모델의 요약 정보 출력
# .summary() 메서드는 모델의 통계적 결과(계수, p-값, R-squared 등)를 자세히 보여줍니다.
print("\n--- 포아송 회귀 모델 학습 결과 요약 ---")
print(poisson_results.summary())

포아송 회귀 모델 학습 중...
Optimization terminated successfully.
         Current function value: 2.414513
         Iterations 5
포아송 회귀 모델 학습 완료!

--- 포아송 회귀 모델 학습 결과 요약 ---
                          Poisson Regression Results                          
Dep. Variable:         accident_count   No. Observations:                  584
Model:                        Poisson   Df Residuals:                      565
Method:                           MLE   Df Model:                           18
Date:                Mon, 02 Jun 2025   Pseudo R-squ.:                 0.03088
Time:                        12:44:31   Log-Likelihood:                -1410.1
converged:                       True   LL-Null:                       -1455.0
Covariance Type:            nonrobust   LLR p-value:                 1.531e-11
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  

### 전체적인 모델의 한계 및 고려 사항
* 낮은 Pseudo R-squ. (0.03088): 이 모델이 사고 건수의 변동을 설명하는 데 기여하는 바가 매우 낮습니다. 즉, 현재 모델이 설명하지 못하는 accident_count의 변동이 훨씬 크다는 뜻입니다. 이는 우리가 고려하지 않은 다른 중요한 변수들이 있거나, 사고 발생 자체가 예측하기 매우 어려운 무작위성이 크다는 것을 의미할 수 있습니다.
* 통계적으로 유의미한 변수가 적음: temperature, is_holiday, weekday_월, weekday_토를 제외한 대부분의 변수들은 유의미한 영향을 미치지 않는 것으로 나타났습니다. 이는 모델의 예측력 향상을 위해 다른 피처를 추가하거나, 현재 피처들을 조합하는 피처 엔지니어링이 필요할 수 있음을 의미합니다.

# 모델 평가

In [8]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np # 예측값에 음수가 나오지 않도록 np.maximum 사용을 위해 임포트

# X_test, y_test는 이전 단계에서 분할된 테스트 데이터입니다.
# linear_model, poisson_results는 이전 단계에서 학습된 모델입니다.
# 데이터와 모델이 정의되어 있는지 다시 확인합니다.
if 'X_test' not in locals() or 'y_test' not in locals():
    raise ValueError("X_test 또는 y_test 데이터가 정의되지 않았습니다. 이전 단계를 먼저 실행해주세요.")
if 'linear_model' not in locals():
    raise ValueError("linear_model이 정의되지 않았습니다. 선형 회귀 모델 학습 단계를 먼저 실행해주세요.")
if 'poisson_results' not in locals():
    raise ValueError("poisson_results가 정의되지 않았습니다. 포아송 회귀 모델 학습 단계를 먼저 실행해주세요.")


print("--- 모델 성능 평가 시작 ---")

# 1. 선형 회귀 모델 예측 및 평가
print("\n[선형 회귀 모델]")
y_pred_linear = linear_model.predict(X_test)

# 선형 회귀는 음수 값을 예측할 수 있습니다. 사고 건수는 음수가 될 수 없으므로,
# 예측값이 음수일 경우 0으로 처리합니다.
y_pred_linear_clipped = np.maximum(0, y_pred_linear)

mse_linear = mean_squared_error(y_test, y_pred_linear_clipped)
mae_linear = mean_absolute_error(y_test, y_pred_linear_clipped)

print(f"  평균 제곱 오차 (MSE): {mse_linear:.2f}")
print(f"  평균 절대 오차 (MAE): {mae_linear:.2f}")

# 2. 포아송 회귀 모델 예측 및 평가
print("\n[포아송 회귀 모델]")
# statsmodels 모델의 예측 시에도 X_test에 상수항을 추가해야 합니다.
X_test_poisson = sm.add_constant(X_test)
X_test_poisson = X_test_poisson[X_train_poisson.columns]
X_test_poisson = X_test_poisson.astype(float)
y_pred_poisson = poisson_results.predict(X_test_poisson)

mse_poisson = mean_squared_error(y_test, y_pred_poisson)
mae_poisson = mean_absolute_error(y_test, y_pred_poisson)

print(f"  평균 제곱 오차 (MSE): {mse_poisson:.2f}")
print(f"  평균 절대 오차 (MAE): {mae_poisson:.2f}")

print("\n--- 모델 성능 평가 완료 ---")

--- 모델 성능 평가 시작 ---

[선형 회귀 모델]
  평균 제곱 오차 (MSE): 7.63
  평균 절대 오차 (MAE): 2.21

[포아송 회귀 모델]
  평균 제곱 오차 (MSE): 7.76
  평균 절대 오차 (MAE): 2.21

--- 모델 성능 평가 완료 ---


### MAE 가 동일 2.21로 실제 사고 건수와 약 2.21 정도 차이가 발생을 의미

# P-value, IRR

In [10]:
# 1. 계수 (coef) 추출
coefficients = poisson_results.params

# 2. IRR (Incidence Rate Ratio) 계산
irr = np.exp(coefficients)

# 3. p-값 (p-value) 추출
p_values = poisson_results.pvalues

# 결과를 DataFrame으로 만들어 보기 좋게 출력
results_df = pd.DataFrame({
    'Coefficient': coefficients,
    'IRR (exp(coef))': irr,
    'P-value': p_values
})

# p-값에 따라 통계적 유의성 표시 (0.05 기준)
results_df['Significance'] = ['***' if p <= 0.001 else '**' if p <= 0.01 else '*' if p <= 0.05 else '' for p in p_values]

print(results_df)

print("\n--- 유의성 표시: *** (p <= 0.001), ** (p <= 0.01), * (p <= 0.05) ---")

                       Coefficient  IRR (exp(coef))        P-value  \
const                     1.885056         6.586727  2.503277e-116   
game_count               -0.008127         0.991906   8.562067e-01   
is_post_season            0.010636         1.010693   9.215603e-01   
temperature               0.007389         1.007416   7.403984e-04   
precipitation            -0.000703         0.999297   5.970975e-01   
snow_depth                0.000904         1.000905   7.236056e-02   
is_holiday               -0.281515         0.754640   2.697863e-03   
sports_types_농구,야구       -0.117360         0.889265   4.714211e-01   
sports_types_야구           0.022529         1.022785   7.255653e-01   
sports_types_없음           0.097546         1.102462   1.826948e-01   
weather_condition_맑음      0.062818         1.064833   1.417349e-01   
weather_condition_비/눈     0.024564         1.024868   5.896321e-01   
weather_condition_흐림     -0.059474         0.942260   3.273843e-01   
weekday_목           